In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
from sklearn import svm

In [ ]:
all_df = pd.read_csv("data/sensor.csv")

In [ ]:
all_df=all_df.drop(['sensor_00'], axis=1)
all_df=all_df.drop(['timestamp'], axis=1)
all_df=all_df.drop(['sensor_01'], axis=1)
all_df=all_df.drop(['sensor_50'], axis=1)
all_df=all_df.drop(['sensor_51'], axis=1)

In [ ]:
all_df['machine_status']= all_df['machine_status'].map({'NORMAL':0, 'BROKEN': 1, 'RECOVERING': 2}).astype(int) 
all_df = all_df.dropna(how='any')

In [ ]:
X=all_df.drop(['machine_status'], axis=1).values.astype('float64')

In [ ]:
Y = all_df['machine_status'].values.astype('int64')
X.shape

In [ ]:
X = StandardScaler().fit_transform(X)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.33, random_state=None) 

In [ ]:
Ytrain[:2]

In [ ]:
Y[:2]

In [ ]:
C = 5.0
classifiers = {
               'RF5': RandomForestClassifier(n_estimators=5),
               'RF50': RandomForestClassifier(n_estimators=50),                
               'tree':DecisionTreeClassifier(criterion='gini',max_depth=5),
                'SVM':svm.SVC(C=5.0,random_state=0, kernel='rbf' ,probability=True)
              }

plt.figure(figsize=(8,8))
n_classifiers = len(classifiers)

In [ ]:
for index, (name, clf) in enumerate(classifiers.items()):
    clf.fit(Xtrain, Ytrain)
    probs = clf.predict_proba(Xtest)
    fpr, tpr, thresholds = roc_curve(Ytest, probs[:, 1])
    roc_auc = auc(fpr, tpr)
    print ('For model', name, 'accuracy =', clf.score(Xtest,Ytest))
    plt.plot(fpr, tpr, label='%s (area = %0.2f)' % (name, roc_auc))
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()